In [ ]:
!pip install modin
!pip install interruptingcow

In [ ]:
from measures import n_1_imb_mean, n_3_imb_mean, tlcm, degOver, degIR, imbalance_ratio
import numpy as np 
from sklearn.utils import Bunch
import modin.pandas as pd 
from tqdm import tqdm
from interruptingcow import timeout
import os
import numpy as np 
from sklearn.utils import Bunch
import modin.pandas as pd 
from tqdm import tqdm
from interruptingcow import timeout
import os
os.cpu_count()

# Loading datasets

In [ ]:
def load_dataset(path_: str) -> Bunch: 
    dataset = np.genfromtxt(path_, skip_header=True, delimiter=",")
    X = dataset[:,0:-1]
    y = dataset[:,-1]
    return Bunch(data=np.array(X), target=y)

In [ ]:

processed_dataset=[]
dataset_name=[]
filepath="Datasets_Numerical/"

import os
datasets = []

datasets = os.listdir("Datasets_Numerical/")


for dataset in datasets:
    if dataset != ".DS_Store":
      dataset_=load_dataset(filepath + dataset)
      processed_dataset.append(dataset_)
      dataset_name.append(dataset)


In [ ]:
len(processed_dataset), len(dataset_name)

In [ ]:
new_list=[]
for i in range(len(processed_dataset)):
  new_list.append([dataset_name[i], processed_dataset[i]])

In [ ]:
len(new_list)

In [ ]:
for i in range(len(processed_dataset)):
  print(new_list[i][0])

# Function for measuring Complexity Score

In [ ]:
def result_df(dataset_): 
    n_1_score = n_1_imb_mean(dataset_) 
    n_3_score = n_3_imb_mean(dataset_)
    tlcm_score = tlcm(dataset_)   
    degOver_score = degOver(dataset_)
    degIR_score = degIR(dataset_)
    ir_score = imbalance_ratio(dataset_)
    return  n_1_score, n_3_score,tlcm_score, degOver_score, degIR_score, ir_score

  

In [ ]:
complexity_measure={}

    
for i in tqdm(range(len(processed_dataset))):
      dataset_name=new_list[i][0]
      dataset_name=dataset_name.replace(".csv","")
      print(f"Current Dataset: {dataset_name}")
      dataset_=new_list[i][1]
      complexity_measure[dataset_name]={}
      n_1_score, n_3_score,tlcm_score,degOver_score, degIR_score, ir_score=result_df(dataset_)
      complexity_measure[dataset_name]["n_1_score"]=n_1_score
      complexity_measure[dataset_name]["n_3_score"]=n_3_score
      complexity_measure[dataset_name]["tlcm_score"]=tlcm_score
      complexity_measure[dataset_name]["degOver_score"]=degOver_score
      complexity_measure[dataset_name]["degIR_score"]=degIR_score
      complexity_measure[dataset_name]["Ir_score"]=ir_score
 

In [ ]:
complexity_measure

In [ ]:
df= pd.DataFrame.from_dict(complexity_measure)

In [ ]:
df.head()

# Store the result and Plotting Graphs

In [ ]:
dataframe =df.transpose()
dataframe.to_csv("final_results.csv")

In [ ]:
dataframe.sort_values(by=['n_1_score', 'n_3_score','tlcm_score'], inplace=True,  ascending=True)
dataframe.head()

In [ ]:
# Plot and compare all of the model results
import matplotlib.pyplot as plt
dataframe.drop(['Ir_score', 'degIR_score'],axis=1).plot(kind="bar",stacked=True, figsize=(9,6)).legend(bbox_to_anchor=(1.0, 1.0),title="Complexity_Score");
# Saving the figure
plt.tight_layout()
plt.savefig("output_bar.jpg", dpi=100)

In [ ]:
# Plot and compare all of the model results
dataframe.drop(['Ir_score', 'degIR_score'],axis=1).plot(kind="barh",stacked=True, figsize=(8,8)).legend(bbox_to_anchor=(1.0, 1.0), title="Complexity_Score");
# Saving the figure
plt.tight_layout()
plt.savefig("output_barh.jpg",dpi=100)
